In [ ]:
from IPython.display import clear_output

!pip install openpyxl
!pip install transformers
!pip install sentencepiece
!pip install trl
clear_output()

In [ ]:
import pandas as pd

df = pd.read_excel('evaluation_results1_test_rejected_answers.xlsx')
df

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "DAMO-NLP-MT/polylm-1.7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
if tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token
else:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model.resize_token_embeddings(len(tokenizer))

In [ ]:
import os
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'

# Initialize the distributed environment
torch.distributed.init_process_group(backend='nccl')

# Wrap the model with DDP
model = model.cuda()
model = DDP(model, device_ids=[torch.cuda.current_device()])
model.config = model.module.config

In [ ]:
from datasets import Dataset

train_dataset = df.to_dict(orient='list')
train_dataset = Dataset.from_dict(train_dataset)
train_dataset

In [ ]:
from trl import DPOTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=10,
)

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)
dpo_trainer.train()

In [ ]:
torch.cuda.device_count()